<a href="https://colab.research.google.com/github/ke-neova/theia/blob/master/Copy_of_Lora_with_Flux_Comfyui_on_Google_Colab_Free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Notes
- Uncheck `UPDATE_COMFY_UI`, and `USE_COMFYUI_MANAGER` and `INSTALL_GGUF_NODE` after first run if `USE_GOOGLE_DRIVE` is checked.


### Install Dependencies

In [ ]:
# #@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
USE_COMFYUI_MANAGER = True  #@param {type:"boolean"}
INSTALL_GGUF_NODE = True  #@param {type:"boolean"}
INSTALL_CUSTOM_NODES_DEPENDENCIES = True  #@param {type:"boolean"}
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
OPTIONS['USE_COMFYUI_MANAGER'] = USE_COMFYUI_MANAGER
OPTIONS['INSTALL_GGUF_NODE'] = INSTALL_GGUF_NODE
OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES'] = INSTALL_CUSTOM_NODES_DEPENDENCIES

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f ".ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/nightly/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/nightly/windows_base_files/run_nvidia_gpu.bat" ] && chmod 755 .ci/nightly/windows_base_files/run_nvidia_gpu.bat
  ![ -f ".ci/update_windows/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat" ] && chmod 755 .ci/update_windows_cu118/update_comfyui_and_python_dependencies.bat
  ![ -f ".ci/update_windows/update.py" ] && chmod 755 .ci/update_windows/update.py
  ![ -f ".ci/update_windows/update_comfyui.bat" ] && chmod 755 .ci/update_windows/update_comfyui.bat
  ![ -f ".ci/update_windows/README_VERY_IMPORTANT.txt" ] && chmod 755 .ci/update_windows/README_VERY_IMPORTANT.txt
  ![ -f ".ci/update_windows/run_cpu.bat" ] && chmod 755 .ci/update_windows/run_cpu.bat
  ![ -f ".ci/update_windows/run_nvidia_gpu.bat" ] && chmod 755 .ci/update_windows/run_nvidia_gpu.bat

  !git pull

!echo -= Install dependencies =-
!pip3 install accelerate
!pip3 install einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip3 install torchsde
!pip3 install kornia>=0.7.1 spandrel soundfile sentencepiece

if OPTIONS['USE_COMFYUI_MANAGER']:
  %cd custom_nodes

  # Correction of the issue of permissions being deleted on Google Drive.
  ![ -f "ComfyUI-Manager/check.sh" ] && chmod 755 ComfyUI-Manager/check.sh
  ![ -f "ComfyUI-Manager/scan.sh" ] && chmod 755 ComfyUI-Manager/scan.sh
  ![ -f "ComfyUI-Manager/node_db/dev/scan.sh" ] && chmod 755 ComfyUI-Manager/node_db/dev/scan.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-linux.sh
  ![ -f "ComfyUI-Manager/scripts/install-comfyui-venv-win.bat" ] && chmod 755 ComfyUI-Manager/scripts/install-comfyui-venv-win.bat

  ![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager
  %cd ComfyUI-Manager
  !git pull


if OPTIONS['INSTALL_GGUF_NODE']:
  %cd $WORKSPACE
  %cd custom_nodes
  !git clone https://github.com/city96/ComfyUI-GGUF
  !pip install -r ComfyUI-GGUF/requirements.txt

%cd $WORKSPACE

if OPTIONS['INSTALL_CUSTOM_NODES_DEPENDENCIES']:
  !echo -= Install custom nodes dependencies =-
  !pip install GitPython
  !python custom_nodes/ComfyUI-Manager/cm-cli.py restore-dependencies





/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-


### Download Resources

In [ ]:
# Download the FLUX model from Hugging Face and save it in the ./models/unet directory
!wget -c "https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf" -P "./models/unet"

# Download the CLIP T5 model from Hugging Face and save it in ./models/clip
!wget -c "https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf" -P "./models/clip"

# Download the safetensors file for the CLIP model
!wget -c "https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors" -P "./models/clip_vision"

# Download the VAE model from Hugging Face and save it in ./models/vae
!wget -c "https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors" -P "./models/vae"
!wget -c "https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors" -P "./models/wan_2.1_vae"

# Download the LoRa model from Civitai.com Replace `xxxxxxxx` with your actual token
!wget -c "https://civitai.com/api/download/models/1529225?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" -O "./models/loras/Flux Skinny Thinspo Petite v3_epoch_5.safetensors"

!wget -c "https://civitai.com/api/download/models/1541625?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/Viking_punk.safetensors"

!wget -c "https://civitai.com/api/download/models/1540666?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/HT-Mavis.safetensors"

!wget -c "https://civitai.com/api/download/models/1549117?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/Orcs.safetensors"

!wget -c "https://civitai.com/api/download/models/1537109?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/Helen Parr.safetensors"

!wget -c "https://civitai.com/api/download/models/1485598?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/Dressed Animals.safetensors"

!wget -c "https://civitai.com/api/download/models/1513808?type=Model&format=SafeTensor&token=3467b9cbf268889f285d335df5e967e1" =O "./models/loras/Medieval_Worlds.safetensors"

--2025-03-28 13:40:30--  https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.121, 13.35.202.40, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/5c/3e5c1f82226fc1460150126b5513fa266f4b8aea7194d1a31c944ad9da31ae10/75bb19459b5240c9f373b5af527584af15b675867fa142efadf7478d6abbf62b?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27flux1-dev-Q4_K_S.gguf%3B+filename%3D%22flux1-dev-Q4_K_S.gguf%22%3B&Expires=1743172830&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzE3MjgzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNlLzVjLzNlNWMxZjgyMjI2ZmMxNDYwMTUwMTI2YjU1MTNmYTI2NmY0YjhhZWE3MTk0ZDFhMzFjOTQ0YWQ5ZGEzMWFlMTAvNzViYjE5NDU5YjUyNDBjOWYzNzNiNWFmNTI3NTg0YWYxNWI2NzU4NjdmYTE0MmVmYWRmNzQ3OGQ2YWJiZjYyYj9y

### Run ComfyUI

In [ ]:
!wget -P ~ https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i ~/cloudflared-linux-amd64.deb
!pip install -r requirements.txt
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2025-03-28 13:41:11--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb [following]
--2025-03-28 13:41:11--  https://github.com/cloudflare/cloudflared/releases/download/2025.2.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/4f284b8f-4452-49e8-a0c0-0e45c3b6e65e?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20250328%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20250328T134112Z&X-Amz-Expires=300&X-Amz-Signature=42a8608d63e6902b153dcb96fd94eff39451b42f7b5c74f1e963c413